In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.appName("Data Exploration") \
.getOrCreate()

25/06/29 04:55:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
!hadoop fs -ls -h /data/olist/

Found 9 items
-rw-r--r--   2 harshit01shy hadoop      8.6 M 2025-06-28 14:56 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 harshit01shy hadoop     58.4 M 2025-06-28 14:56 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 harshit01shy hadoop     14.7 M 2025-06-28 14:56 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 harshit01shy hadoop      5.5 M 2025-06-28 14:57 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 harshit01shy hadoop     13.8 M 2025-06-28 14:57 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 harshit01shy hadoop     16.8 M 2025-06-28 14:57 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 harshit01shy hadoop      2.3 M 2025-06-28 14:57 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 harshit01shy hadoop    170.6 K 2025-06-28 14:57 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 harshit01shy hadoop      2.6 K 2025-06-28 14:57 /data/olist/product_category_name_translation.csv


In [5]:
hdfs_path = "/data/olist/"

In [7]:
#Read All DATASETS...

Customer_df = spark.read.csv(hdfs_path + "olist_customers_dataset.csv",header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + "olist_geolocation_dataset.csv",header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + "olist_order_items_dataset.csv",header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + "olist_order_payments_dataset.csv",header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + "olist_order_reviews_dataset.csv",header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + "olist_orders_dataset.csv",header=True,inferSchema=True)
products_df= spark.read.csv(hdfs_path + "olist_products_dataset.csv",header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + "olist_sellers_dataset.csv",header=True,inferSchema=True)
category_name_df = spark.read.csv(hdfs_path + "product_category_name_translation.csv",header=True,inferSchema=True)

In [8]:
geolocation_df.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [9]:
Customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [10]:
# Data Lekage or Drop

print(f"Customers : {Customer_df.count()} rows")
print(f"geolocation : {geolocation_df.count()} rows")
print(f"order_items : {order_item_df.count()} rows")
print(f"order_payments : {payments_df.count()} rows")
print(f"order_reviews : {reviews_df.count()} rows")
print(f"orders : {orders_df.count()} rows")
print(f"products : {products_df.count()} rows")
print(f"sellers : {sellers_df.count()} rows")
print(f"product_category : {category_name_df.count()} rows")

Customers : 99441 rows


geolocation : 1000163 rows
order_items : 112650 rows
order_payments : 103886 rows
order_reviews : 104162 rows
orders : 99441 rows
products : 32951 rows
sellers : 3095 rows
product_category : 71 rows


In [14]:
from pyspark.sql.functions import col,when,count

#Check for nulls in Critical Fields

Customer_df.select([count(when(col(c).isNull(),1)).alias(c) for c in Customer_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [15]:
#Check Duplicate Values in Customer

Customer_df.groupBy("customer_id").count().filter('count>1').show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [16]:
#Customer Distribution By State

Customer_df.groupBy("customer_state").count().orderBy('count',ascending=False).show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [18]:
orders_df.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [19]:
#Orders Distribution Status:----->>>>

orders_df.groupBy('order_status').count().orderBy('count',ascending=False).show()


+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [26]:
#Average Delivery Time Analysis

delivery_df = orders_df.select('order_id','order_purchase_timestamp','order_delivered_customer_date')
delivery_df.show(10)

+--------------------+------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|
+--------------------+------------------------+-----------------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|
|a4591c265e18cb1dc...|     2017-07-09 21:57:05|          2017-07-26 10:57:55|
|136cce7faa42fdb2c...|     2017-04-11 12:22:08|                         NULL|
|6514b8ad8028c9f2c...|     2017-05-16 13:10:30|          2017-05-26 12:55:51|
|76c6e866289321a7c...|     2017-01-23 18:29:09|          2017-02-02 14:08:10|
|e69bfb5eb88e0ed6a...|     2017-07-29 11:55:02|          2017-08

In [30]:
from pyspark.sql.functions import datediff,to_date

delivery_detail_df = delivery_df.withColumn('delivery_time',datediff(col('order_delivered_customer_date'),col('order_purchase_timestamp')))

In [31]:
delivery_detail_df.orderBy('delivery_time',ascending=False).show()

+--------------------+------------------------+-----------------------------+-------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|delivery_time|
+--------------------+------------------------+-----------------------------+-------------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|          210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|          208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|          196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|          195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|          195|
|0f4519c5f1c541dde...|     2017-03-09 13:26:57|          2017-09-19 14:38:21|          194|
|47b40429ed8cce3ae...|     2018-01-03 09:44:01|          2018-07-13 20:51:31|          191|
|2fe324febf907e3ea...|     2017-03-13 20:17:10|          2017-09-19 17:00:07|   

In [20]:
# Checking which payment type is used more by customers. 

payments_df.groupBy('payment_type').count().orderBy('count',ascending=False).show()

+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [24]:
order_item_df.show(20)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [21]:
from pyspark.sql.functions import sum
top_products = order_item_df.groupBy('product_id').agg(sum('price').alias('total_sales'))
top_products.orderBy('total_sales',ascending=False).show()

+--------------------+------------------+
|          product_id|       total_sales|
+--------------------+------------------+
|bb50f2e236e5eea01...|           63885.0|
|6cdd53843498f9289...| 54730.20000000005|
|d6160fb7873f18409...|48899.340000000004|
|d1c427060a0f73f6b...| 47214.51000000006|
|99a4788cb24856965...|43025.560000000085|
|3dd2a17168ec895c7...| 41082.60000000005|
|25c38557cf793876c...| 38907.32000000001|
|5f504b3a1c75b73d6...|37733.899999999994|
|53b36df67ebb7c415...| 37683.42000000001|
|aca2eb7d00ea1a7b8...| 37608.90000000007|
|e0d64dcfaa3b6db5c...|          31786.82|
|d285360f29ac7fd97...|31623.809999999983|
|7a10781637204d8d1...|           30467.5|
|f1c7f353075ce59d8...|          29997.36|
|f819f0c84a64f02d3...|29024.479999999996|
|588531f8ec37e7d5f...|28291.989999999998|
|422879e10f4668299...|26577.219999999972|
|16c4e87b98a9370a9...|           25034.0|
|5a848e4ab52fd5445...|24229.029999999962|
|a62e25e09e05e6faf...|           24051.0|
+--------------------+------------